In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# load packages
import sys # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import numpy as np # scientific computing
print("NumPy version: {}". format(np.__version__))

import pandas as pd # data processing and analysis 
print("pandas version: {}". format(pd.__version__))

import matplotlib
import matplotlib.pyplot as plt # plotting
%matplotlib inline 
print("matplotlib version: {}". format(matplotlib.__version__))

import seaborn as sns
print("seaborn version: {}". format(sns.__version__))
 
import sklearn # machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

from itertools import product

import time
import pickle

print("----------")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read csv file into data frames
df_items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
df_shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
df_sales_raw = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
df_sales = df_sales_raw.copy() # make a copy of the raw data
df_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
df_item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
df_sales.tail(3)

# Data Cleanup

## Shops
Some shops are duplicates according to their shop name and the time when they have sales. Luckily, only one of the duplicated shop_ids is in the test set. The shop_id from the test set will remain.


In [ ]:
# convert data to datetime object
df_sales['date'] = pd.to_datetime(df_sales['date'], dayfirst=True)

In [ ]:
# check minimum and maximum sales date of the shops with similar names
print("Sales period shop_id =1: ", df_sales[df_sales.shop_id==1]['date'].min(), df_sales[df_sales.shop_id==1]['date'].max())
print("Sales period shop_id =58: ", df_sales[df_sales.shop_id==58]['date'].min(), df_sales[df_sales.shop_id==58]['date'].max())
print("-----")
print("Sales period shop_id =0: ", df_sales[df_sales.shop_id==0]['date'].min(), df_sales[df_sales.shop_id==0]['date'].max())
print("Sales period shop_id =57: ", df_sales[df_sales.shop_id==57]['date'].min(), df_sales[df_sales.shop_id==57]['date'].max())
print("-----")
print("Sales period shop_id =10: ", df_sales[df_sales.shop_id==10]['date'].min(), df_sales[df_sales.shop_id==10]['date'].max())
print("Sales period shop_id =11: ", df_sales[df_sales.shop_id==11]['date'].min(), df_sales[df_sales.shop_id==11]['date'].max())
print("----- 10 has a sales hole that is filled by 11")

In [ ]:
# make duplicated shops one shop, use the shop_id present in the test set (df_test[df_test.shop_id==0].shape[0] -> 0)
# in df_sales
# Жуковский ул. Чкалова 39м² -> Жуковский ул. Чкалова 39м?
df_sales.loc[df_sales.shop_id == 11, 'shop_id'] = 10
# !Якутск ТЦ "Центральный" фран -> Якутск ТЦ "Центральный"
df_sales.loc[df_sales.shop_id == 1, 'shop_id'] = 58
# !Якутск Орджоникидзе, 56 фран -> Якутск Орджоникидзе, 56
df_sales.loc[df_sales.shop_id == 0, 'shop_id'] = 57
# remove from df_shops as well
df_shops.drop([0,1,11], axis=0, inplace=True)

## Items

In [ ]:
# boxplot of item_price: the vast majority of item_price is below 40000. One outlier with over 300000.
plt.figure(figsize=(20,3))
sns.boxplot(x=['item_price'], data=df_sales)
plt.show()

In [ ]:
# item_price
print("Minimum item price: ", df_sales.item_price.min())
print("Maximum item price: ", df_sales.item_price.max())
item_price_max = df_sales.loc[df_sales.item_price == df_sales.item_price.max()]
#df_items[df_items.item_id==item_id_maximum_price]
print("\nRows with maximum item price:\n ",item_price_max)
print("\nItem with highest price:\n ", df_items[df_items.item_id==6066]) # dirty, dirty, dirty, improve later
# this is a software for remotecontrolling PCs called Radmin3, it was sold for 522 persons, high price could be plausible
item_price_min = df_sales[df_sales.item_price == df_sales.item_price.min()]
print("\nRows with minimum item price:\n ",item_price_min)
# there is just one row with an item_price below 0, remove it
df_sales = df_sales[df_sales.item_price > 0]

In [ ]:
# let's check sales of price outlier - it has been sold only once and is not included in the test set. remove it
#df_sales[df_sales.item_id == 6066]
#df_test[df_test.item_id == 6066]
df_sales = df_sales[df_sales.item_price < 100000]

In [ ]:
# boxplot of item_cnt_day: the vast majority of item_cnt_day is around 0-100. A few negative values show up (returns) and 2 outliers (approx 1000/2200)
plt.figure(figsize=(20,3))
sns.boxplot(x=['item_cnt_day'], data=df_sales)
plt.show()

In [ ]:
print("Minimum item_cnt_day: ", df_sales.item_cnt_day.min())
print("Maximum item_cnt_day: ", df_sales.item_cnt_day.max())

item_cnt_day_max = df_sales[df_sales.item_cnt_day == df_sales.item_cnt_day.max()]
print("\nRows with maximum daily sales amount:\n ",item_cnt_day_max)
print("\nItem sold most often:\n ", df_items[df_items.item_id==11373]) # dirty, dirty, dirty, improve later
# looks like this is shipping cost. It is only present for shop_id 12 which seems to be the online shop

item_cnt_day_b0 = df_sales[df_sales.item_cnt_day < 0]
print("\nNumber of Rows with sales below 0, i.e. returns: ",item_cnt_day_b0.shape[0])
print("Percentage of Rows with sales below 0, i.e. returns: % 1.2f " %(item_cnt_day_b0.shape[0] /df_sales.shape[0]*100) )
# the percentage of rows with returns is low, but before discarding them, let's look again after monthly aggregation of data. 

In [ ]:
# get the second highest sold item
#df_sales.item_cnt_day.sort_values(ascending=False)[0:3]
#df_sales.loc[2326930]
#df_items[df_items.item_id == 20949]
# 2nd highest sold item, Фирменный пакет майка 1С Интерес белый (34*42).., plastic bag?

In [ ]:
# let's quickly look at how the sales of the 2 highest sold items looks over time: item_id 11373 and 20949
item11373 = df_sales[df_sales.item_id == 11373]
item20949 = df_sales[df_sales.item_id == 20949]
fig = plt.figure() # create figure
ax0 = fig.add_subplot(1, 2, 1) # add subplot 1 (1 row, 2 columns, first plot)
ax1 = fig.add_subplot(1, 2, 2) 
item11373.plot(x='date',y='item_cnt_day', figsize=(20,4), ax=ax0)
item20949.plot(x='date',y='item_cnt_day', figsize=(20,4), ax=ax1)
ax0.set_title("Item 11373: no of items sold over time")
ax1.set_title("Item 20949: no of items sold over time")
plt.show()
# it can be seen that the high sales is only once. I decide to drop this row.

In [ ]:
# everything with sales 1000 and bigger is declared as outlier based on the investigations before and removed
df_sales = df_sales[df_sales.item_cnt_day < 1000]

# Data Aggregation to monthly values

Like shown in Programming Assignment week3. The column with monthly aggregation is called 'target'. Could have also been called 'item_cnt_month'.

In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in df_sales['date_block_num'].unique():
    cur_shops = df_sales[df_sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = df_sales[df_sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
#gb = df_sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum','item_price':'mean'})
gb = df_sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':'sum'})
#gb = gb.rename(columns={'item_cnt_day':'target', 'item_price':'mean_item_price'})
gb = gb.rename(columns={'item_cnt_day':'target'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

# this grid has lots of 0 target values, approx 85%. 
# all shop_ids are combined with all item_ids from one month. If an item has been sold in a month but from another shop, all shops
# where the item has not been sold will get a target of 0.

In [ ]:
all_data.head(3)

In [ ]:
# downcast data type for better memory usage
all_data['shop_id'] = all_data['shop_id'].astype(np.int8)
all_data['item_id'] = all_data['item_id'].astype(np.int16)
all_data['date_block_num'] = all_data['date_block_num'].astype(np.int8)

Now let's look again at the monthly items sold.

In [ ]:
# boxplot of monthly item sold sum: the biggest outlier is still there. It must have only been sold once.
plt.figure(figsize=(20,3))
sns.boxplot(x=['target'], data=all_data)
plt.show()

In [ ]:
all_data[all_data.target < 0]

In [ ]:
all_data[all_data.target > 1500]

After aggregation there are still negative item counts.
BUT: in the final submission the target values have to be clipped to be in 0-20 range. I wonder if the right place for clipping is before modelling or after. Gordon Henderson suggests in the forum to do it after feature engineering, just before modelling. dlarionov clips once just after building the grid, and again before submission.

I decidied to clip before modelling now. And again before submission. keep the higher numbers for feature engineering.

In [ ]:
all_data.target.describe()

# Data Preprocessing
## Monthly info

In [ ]:
# create data frame to store all monthly values
# first get the number of transactions that have been performed in each month
month_info = df_sales.date_block_num.value_counts()
month_info = month_info.to_frame(name='monthly_transactions').reset_index()
month_info = month_info.rename(columns={'index':'date_block_num'})
month_info = month_info.sort_values(by='date_block_num', ascending=True)

# then get the total items sold each month
#month_sum_clipped = all_data.groupby('date_block_num')['target'].sum()
month_sum = df_sales.groupby('date_block_num')['item_cnt_day'].sum()
month_info=pd.merge(month_info,month_sum.to_frame(), on='date_block_num')
#month_info=pd.merge(month_info,month_sum_clipped.to_frame(), on='date_block_num')
month_info = month_info.rename(columns={'item_cnt_day':'month_sum'})
#month_info = month_info.rename(columns={'target':'month_clip_sum'})

# add the number of days per month
days_per_month = [31,28,31,30,31,30,31,31,30,31,30,31,
                  31,28,31,30,31,30,31,31,30,31,30,31,
                 31,28,31,30,31,30,31,31,30,31]
month_info['days_per_month'] = days_per_month

# add number of month: 1= january and so on
month = [1,2,3,4,5,6,7,8,9,10,11,12,
        1,2,3,4,5,6,7,8,9,10,11,12,
        1,2,3,4,5,6,7,8,9,10]
month_info['month'] = month

month_info.tail()

In [ ]:
# month_info.plot(x='date_block_num', y=['monthly_transactions','month_sum', 'month_clip_sum'], figsize=(10,6))
month_info.plot(x='date_block_num', y=['monthly_transactions','month_sum'], figsize=(10,6))
plt.show()

In [ ]:
# downcast data type for better memory usage
month_info['date_block_num'] = month_info['date_block_num'].astype(np.int8)
month_info['monthly_transactions'] = month_info['monthly_transactions'].astype(np.int32)
month_info['days_per_month'] = month_info['days_per_month'].astype(np.int8)
month_info['month'] = month_info['month'].astype(np.int8)

## Shop Data

In [ ]:
# extract city name
df_shops['city']= df_shops.shop_name.str.split(' ', expand=True)[0]
# change city to 'other' for shop_ids 55, 12 and 9. Their translated names are Digital Warehouse 1C-Online, Online shop Emergency and Outbound Trade.
df_shops.loc[df_shops.shop_id == 55, 'city'] = 'other'
df_shops.loc[df_shops.shop_id == 12, 'city'] = 'other'
df_shops.loc[df_shops.shop_id == 9, 'city'] = 'other'

In [ ]:
# downcast data type for better memory usage
df_shops['shop_id'] = df_shops['shop_id'].astype(np.int8)

## Item Category Data

In [ ]:
df_item_cat['item_broad_cat']= df_item_cat.item_category_name.str.split(' - ', expand=True)[0]
# correct sub category names and set those with fewer thatn 3 occurances to 'other'
df_item_cat.loc[26:31, 'item_broad_cat'] = 'Игры'
df_item_cat.loc[32, 'item_broad_cat'] = 'Карты оплаты'
df_item_cat.loc[0, 'item_broad_cat'] = 'other'
df_item_cat.loc[8:9, 'item_broad_cat'] = 'other'
df_item_cat.loc[79:83, 'item_broad_cat'] = 'other'
df_item_cat.head()

In [ ]:
#pd.set_option('display.max_rows', 500)
df_item_cat["split"] = df_item_cat.item_category_name.apply(lambda x: x.split("-"))
df_item_cat["item_sub_cat"] = df_item_cat.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
df_item_cat = df_item_cat.drop(columns='split')

# Generate Output

In [ ]:
all_data.to_pickle('all_data_cleaned.pkl') # use pickle because reading/writing is faster than to csv
df_shops.to_csv('df_shops_cleaned.csv')
month_info.to_pickle('month_info.pkl')
df_item_cat.to_csv('df_item_cat_cleaned.csv')

In [ ]:
# all_data = pd.read_pickle('all_data_cleaned.pkl')

..

..

..

..
..
..
..
..
..
..
..
Data Preprocessing based on item_name - discontinued, too much effort for uncertain return

In [ ]:
df_item_test = pd.merge(df_items, df_item_cat, on='item_category_id', how='left')
df_item_test

In [ ]:
# not continued here, seen as too much effort for uncertain return
def create_sub_cat (row):
    if '1С:Аудиокниги' in row['item_name']:
        return 'Audiobooks'
    if '1С:Аудио'  in row['item_name']:
        return 'Audio'
    if '1С:Аудиотеатр' in row['item_name']:
        return 'Audiotheater'
    if '1С:Образовательная' in row['item_name']:
        return 'Educational'
    if '1С:Познавательная' in row['item_name']:
        return 'Cognitive'
    if '1С:Предпр' in row['item_name']:
        return 'Enterprise'
    if '1С:Репетитор' in row['item_name']:
        return 'Tutor'
    if '1С:Предпр' in row['item_name']:
        return 'Enterprise'
    if '1С:Управл' in row['item_name']:
        return 'Management'
    if '1С:Школа' in row['item_name']:
        return 'School'
    if 'обучение' in row['item_name']:
        return 'School'
    if '3D Action Puzzle' in row['item_name']:
        return '3D Action Puzzle'
    if '3D Crystal Puzzle' in row['item_name']:
        return '3D Crystal Puzzle'
    if '3D Puzzle' in row['item_name']:
        return '3D Puzzle'
    if '3D Сфера-Пазлы' in row['item_name']:
        return '3D Sphere Puzzle'
    if '4D пазлы' in row['item_name']:
        return '4D Puzzle'
    if '3D Сфера-Пазлы' in row['item_name']:
        return '3D Sphere Puzzle'
    if '3D Сфера-Пазлы' in row['item_name']:
        return '3D Sphere Puzzle'
    if '3D Сфера-Пазлы' in row['item_name']:
        return '3D Sphere Puzzle'
    if '3D Сфера-Пазлы' in row['item_name']:
        return '3D Sphere Puzzle'
    return 'Other'
   
# how to use
#col = df_item_test.apply (lambda row: create_sub_cat(row), axis=1)
#df_item_test = df_item_test.assign(sub_cat=col.values) # assign values to column 'sub_cat'
#df_item_test[110:120]